In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from xgboost.sklearn import XGBClassifier
from sklearn.grid_search import GridSearchCV   #Perforing grid search
import matplotlib as plt
import itertools

RuntimeError: module compiled against API version a but this version of numpy is 9

In [2]:
#Read Datasets
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
y_train=train["ACTION"]
target='ACTION'

ID=test['id']
test=test.drop(['id'],axis=1)
train.info()
#test=test.drop(['id'],axis=1)
#len(np.unique(train["ROLE_FAMILY_DESC"]))


<class 'pandas.core.frame.DataFrame'>
Int64Index: 32769 entries, 0 to 32768
Data columns (total 10 columns):
ACTION              32769 non-null int64
RESOURCE            32769 non-null int64
MGR_ID              32769 non-null int64
ROLE_ROLLUP_1       32769 non-null int64
ROLE_ROLLUP_2       32769 non-null int64
ROLE_DEPTNAME       32769 non-null int64
ROLE_TITLE          32769 non-null int64
ROLE_FAMILY_DESC    32769 non-null int64
ROLE_FAMILY         32769 non-null int64
ROLE_CODE           32769 non-null int64
dtypes: int64(10)
memory usage: 2.8 MB


In [24]:
rows=train.shape[0]
train=train.drop('ACTION',axis=1)
result = pd.concat([train,test])
for dfgs in predictors:
    affiliate_channel_maxs = []
    affiliate_channel_maxs_dict = {}
    affiliate_channel_maxs = list(enumerate(np.unique(result[dfgs])))
    affiliate_channel_maxs_dict = {name : i for i, name in affiliate_channel_maxs}
    result[dfgs+'_enum'] = result[dfgs].map(lambda x: affiliate_channel_maxs_dict[x]).astype(int)
new_train=result[:rows]
new_train[target]=y_train
new_test=result[rows:]
new_train['combination']=new_train['MGR_ID_enum']*10000 + new_train['RESOURCE_enum']
new_test['combination']=new_test['MGR_ID_enum']*10000 + new_test['RESOURCE_enum']
new_train.info()

/Users/shashank/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/shashank/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/shashank/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32769 entries, 0 to 32768
Data columns (total 74 columns):
RESOURCE                              32769 non-null int64
MGR_ID                                32769 non-null int64
ROLE_ROLLUP_1                         32769 non-null int64
ROLE_ROLLUP_2                         32769 non-null int64
ROLE_DEPTNAME                         32769 non-null int64
ROLE_TITLE                            32769 non-null int64
ROLE_FAMILY_DESC                      32769 non-null int64
ROLE_FAMILY                           32769 non-null int64
ROLE_CODE                             32769 non-null int64
ROLE_FAMILYROLE_TITLE                 32769 non-null float64
ROLE_FAMILYRESOURCE                   32769 non-null float64
ROLE_FAMILYROLE_CODE                  32769 non-null float64
ROLE_FAMILYROLE_ROLLUP_1              32769 non-null float64
ROLE_FAMILYROLE_DEPTNAME              32769 non-null float64
ROLE_FAMILYROLE_ROLLUP_2              32769 non-null fl

In [4]:
target='ACTION'
predictors = [x for x in train.columns if x not in target]
predictors

['RESOURCE',
 'MGR_ID',
 'ROLE_ROLLUP_1',
 'ROLE_ROLLUP_2',
 'ROLE_DEPTNAME',
 'ROLE_TITLE',
 'ROLE_FAMILY_DESC',
 'ROLE_FAMILY',
 'ROLE_CODE']

In [32]:
#new_test=new_test.fillna(0)
#new_test.info()
new_train.to_csv("new_train.csv")
new_test.to_csv("new_test.csv")

In [ ]:
log_model = LogisticRegression(class_weight='balanced')
log_model.fit(train[predictors],train[target])
log_model.score(train[predictors],train[target])
# ID=test['id']
# test=test.drop(['id'],axis=1)

In [ ]:
test=test.fillna(value=0)
predicted = log_model.predict_proba(test)[:,1]
#np.unique(predicted)
preds = pd.DataFrame({"Id":ID,"ACTION":predicted})
preds = preds.set_index('Id')
preds.to_csv('logistic.csv')

In [ ]:
#1-0 Encoding
for f in train.columns:
    print f
    df_all_dummy = pd.get_dummies(train[f], prefix=f)
    train = train.drop([f], axis=1)
    train = pd.concat((train, df_all_dummy), axis=1)
train.head()

In [ ]:
#1-0 Encoding
test=test.drop(['id'],axis=1)
for f in test.columns:
    print f
    df_all_dummy = pd.get_dummies(test[f], prefix=f)
    test = test.drop([f], axis=1)
    test = pd.concat((test, df_all_dummy), axis=1)
test.head()

In [ ]:
#Copy of data
test_back=test
train_back=train

In [ ]:
#XGboost

# gbm = xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05).fit(train, y_train)
param = {'max_depth':4, 'eta':1, 'silent':1, 'objective':'binary:logistic'}
num_round = 2
dtr=xgb.DMatrix(train,label=y_train)
gb = xgb.cv(params=param, dtrain=dtr, num_boost_round = num_round, nfold= 5)
gb.shape[0]
#predicted = gbm.predict(test)

In [ ]:
#Learning Rate
def modelfit(alg, dtrain, predictors,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    feat=[]
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics=["auc"] , early_stopping_rounds=early_stopping_rounds, show_progress=False)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain[target],eval_metric='auc')
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
        
    #Print model report:
    print "\nModel Report"
    print "Accuracy : %.4g" % metrics.accuracy_score(dtrain[target].values, dtrain_predictions)
    print "AUC Score (Train): %f" % metrics.roc_auc_score(dtrain[target], dtrain_predprob)
                    
    feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
    print feat_imp
    #feat.append(feat_imp)
    #return feat
    #.plot(kind='bar', title='Feature Importances')
    #plt.ylabel('Feature Importance Score')

In [20]:
xgb1 = XGBClassifier(
 learning_rate =0.3,
 n_estimators=120,
 max_depth=18,
 min_child_weight=6,
 gamma=0.2,
 reg_alpha=0.2,
 reg_lambda=0.2,
 subsample=0.9,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 #nthread=4,
 scale_pos_weight=1,
 seed=0)
#modelfit(xgb1, train, predictors)
xgb1

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0.2, learning_rate=0.3, max_delta_step=0, max_depth=18,
       min_child_weight=6, missing=None, n_estimators=120, nthread=-1,
       objective='binary:logistic', reg_alpha=0.2, reg_lambda=0.2,
       scale_pos_weight=1, seed=1234567890, silent=True, subsample=0.9)

In [ ]:
clf = GridSearchCV(
    xgb1,
    {
        #'learning_rate':[i/10.0 for i in range(1,4)],
         #'min_child_weight': [5,6,7,8],
         #'max_depth': [10,13,14,18,20],
         #'n_estimators': range(15,18),
        #'gamma':[i/10.0 for i in range(2,4)],
        #'reg_alpha':[0.2,0.3,0.4,0.5],
        #'reg_lambda':[0.3,0.4,0.5,0.6]
        #'colsample_bytree': [0.6,0.7,0.8,0.85,0.9],
    },
    cv=5,
    n_jobs=1,
    verbose=5,
    scoring='roc_auc'
)
clf

In [ ]:
clf.fit(train[predictors],train[target])

In [ ]:
param_test1 = { 'max_depth':range(3,7,2),'min_child_weight':range(1,4,2)}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.5, n_estimators=15, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=0), 
 param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

gsearch1.fit(train[predictors],train[target])
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

In [ ]:
print(clf.best_params_) 
print(clf.best_score_)

In [ ]:
test.info()

In [21]:
# params = {}
# params["objective"] = "binary:logistic"
# params["n_estimators"] = "17"
# params["eval_metric"] = "auc"
# params["min_child_weight"] = 6
# params["max_depth"] = 18
# params["learning_rate"] =0.3
# params['gamma']=0.2,

# params['subsample']=0.8,
# params["colsample_bytree"]= 0.8
# params['nthread']=4,
# params['scale_pos_weight']=1,

#test=test.drop(['id'],axis=1)
#xg_dtst = xgb.DMatrix(test)
#xg_dtrn = xgb.DMatrix(train[predictors], label=y_train)
#model = xgb.train(list(params.items()), xg_dtrn)
#test.info()
#predicted = model.predict(xg_dtst)

# train=train.drop(['MGR_ID'],axis=1)
# test=test.drop(['MGR_ID'],axis=1)
# predictors.remove('MGR_ID')

model = xgb1.fit(train[predictors], train['ACTION'], eval_metric='auc')
predict_ac = xgb1.predict(test[predictors])
predicted = xgb1.predict_proba(test[predictors])[:,1]


In [22]:
# Write Submission
preds = pd.DataFrame({"Id":ID,"ACTION":predicted})
preds = preds.set_index('Id')
preds.to_csv('xgboost.csv')

In [ ]:
poly = PolynomialFeatures(2)
new_train=pd.DataFrame(poly.fit_transform(train[predictors]))
new_train[target]=train[target]
new_train.head()

In [ ]:
new_test=pd.DataFrame(poly.fit_transform(test))
new_test.head()

In [5]:
#Value_Counts 2-way
num=len(train.index)
#removed1=['ROLE_TITLE','ROLE_FAMILY']
#removed2=['ROLE_ROLLUP_1','ROLE_ROLLUP_2']
way2iter=set(train.columns).difference(set(['ACTION']))
for i,j in itertools.combinations(way2iter,2):
    print i,j
    #if ((i in removed1) & (j=='ROLE_CODE')) or ((i=='ROLE_CODE') & (j in removed2)):
    #    continue
    new=dict(train.groupby([i,j])[j].count())
    train[i+j]=pd.Series(zip(train[i],train[j])).map(new)
    test[i+j]=pd.Series(zip(test[i],test[j])).map(new)
#train.head()
# way3lis=[("RESOURCE", "MGR_ID", "ROLE_ROLLUP_2"), 
# ("RESOURCE", "MGR_ID", "ROLE_DEPTNAME"),
# ("RESOURCE", "MGR_ID", "ROLE_CODE"),
# ("RESOURCE", "MGR_ID", "ROLE_FAMILY_DESC"),
# ("RESOURCE", "MGR_ID", "ROLE_FAMILY"),
# ("ROLE_FAMILY", "MGR_ID", "ROLE_DEPTNAME"),
# ("ROLE_FAMILY", "MGR_ID", "ROLE_ROLLUP_2"),
# ("RESOURCE", "ROLE_DEPTNAME", "ROLE_FAMILY"),
# ("RESOURCE", "ROLE_DEPTNAME", "ROLE_ROLLUP_2")]
#3way
for (i,j,k) in itertools.combinations(way2iter,3):
    print i,j,k
    #if (((i in removed1) & (j=='ROLE_CODE')) or ((i=='ROLE_CODE') & (j in removed2))) or (((j in removed1) & (k=='ROLE_CODE')) or ((j=='ROLE_CODE') & (k in removed2))):
    #    continue
    new=dict(train.groupby([i,j,k])[j].count())
    train[i+j+k]=pd.Series(zip(train[i],train[j],train[k])).map(new)
    test[i+j+k]=pd.Series(zip(test[i],test[j],test[k])).map(new)
#train.head()

#Value_Counts 1-way
for i in way2iter:
    counts_tot=dict(train[i].value_counts())
    train[i+'count']=train[i].map(counts_tot)
    test[i+'count']=test[i].map(counts_tot)
    test=test.fillna(value=0)


#train=train.drop(["ROLE_ROLLUP_1","ROLE_ROLLUP_2"],axis=1)
#test=test.drop(["ROLE_ROLLUP_1","ROLE_ROLLUP_2"],axis=1)

ROLE_FAMILY ROLE_TITLE
ROLE_FAMILY RESOURCE
ROLE_FAMILY ROLE_CODE
ROLE_FAMILY ROLE_ROLLUP_1
ROLE_FAMILY ROLE_DEPTNAME
ROLE_FAMILY ROLE_ROLLUP_2
ROLE_FAMILY MGR_ID
ROLE_FAMILY ROLE_FAMILY_DESC
ROLE_TITLE RESOURCE
ROLE_TITLE ROLE_CODE
ROLE_TITLE ROLE_ROLLUP_1
ROLE_TITLE ROLE_DEPTNAME
ROLE_TITLE ROLE_ROLLUP_2
ROLE_TITLE MGR_ID
ROLE_TITLE ROLE_FAMILY_DESC
RESOURCE ROLE_CODE
RESOURCE ROLE_ROLLUP_1
RESOURCE ROLE_DEPTNAME
RESOURCE ROLE_ROLLUP_2
RESOURCE MGR_ID
RESOURCE ROLE_FAMILY_DESC
ROLE_CODE ROLE_ROLLUP_1
ROLE_CODE ROLE_DEPTNAME
ROLE_CODE ROLE_ROLLUP_2
ROLE_CODE MGR_ID
ROLE_CODE ROLE_FAMILY_DESC
ROLE_ROLLUP_1 ROLE_DEPTNAME
ROLE_ROLLUP_1 ROLE_ROLLUP_2
ROLE_ROLLUP_1 MGR_ID
ROLE_ROLLUP_1 ROLE_FAMILY_DESC
ROLE_DEPTNAME ROLE_ROLLUP_2
ROLE_DEPTNAME MGR_ID
ROLE_DEPTNAME ROLE_FAMILY_DESC
ROLE_ROLLUP_2 MGR_ID
ROLE_ROLLUP_2 ROLE_FAMILY_DESC
MGR_ID ROLE_FAMILY_DESC
ROLE_FAMILY ROLE_TITLE RESOURCE
ROLE_FAMILY ROLE_TITLE ROLE_CODE
ROLE_FAMILY ROLE_TITLE ROLE_ROLLUP_1
ROLE_FAMILY ROLE_TITLE ROLE_DEPTNA

In [7]:
train['combination']=train['MGR_ID']*10000 + train['RESOURCE']
test['combination']=test['MGR_ID']*10000 + test['RESOURCE']

In [13]:
train['combination2']=train['MGR_ID']*10000 + train['ROLE_FAMILY_DESC']
test['combination2']=test['MGR_ID']*10000 + test['ROLE_FAMILY_DESC']

In [14]:
train.shape

(32769, 141)

In [ ]:
new[(56723, 75078)]

In [ ]:
train['MGR_ID'][0]

In [ ]:
zip(train['MGR_ID'],train['RESOURCE'])

In [25]:
from __future__ import division
from collections import defaultdict
from glob import glob
import sys
import math
import os

glob_files = os.getcwd()+'/En/*.csv'
print glob(glob_files)
loc_outfile = 'out.csv'

def kaggle_bag(glob_files, loc_outfile, weights=[], method="average"):
  if method == "average":
    scores = defaultdict(float)
  with open(loc_outfile,"wb") as outfile:
    for i, glob_file in enumerate( glob(glob_files) ):
      print "parsing:", glob_file
      # sort glob_file by first column, ignoring the first line
      lines = open(glob_file).readlines()
      lines = [lines[0]] + sorted(lines[1:])
      for e, line in enumerate( lines ):
        if i == 0 and e == 0:
          outfile.write(line)
        if e > 0:
          row = line.strip().split(",")
          if scores[(e,row[0])] == 0:
            scores[(e,row[0])] = 1
          scores[(e,row[0])] *= float(row[1])
    for j,k in sorted(scores):
      outfile.write("%s,%f\n"%(k,math.pow(scores[(j,k)],1/(i+1))))
    print("wrote to %s"%loc_outfile)

kaggle_bag(glob_files, loc_outfile)

['/Users/shashank/Downloads/PS 3/En/blendingRF,XG,ET,XG,GB.csv', '/Users/shashank/Downloads/PS 3/En/xgboost140feat_all_way_no_enum.csv']
parsing: /Users/shashank/Downloads/PS 3/En/blendingRF,XG,ET,XG,GB.csv
parsing: /Users/shashank/Downloads/PS 3/En/xgboost140feat_all_way_no_enum.csv
wrote to out.csv


In [43]:
from __future__ import division
from collections import defaultdict
from glob import glob
import sys
import math
import os

def kaggle_bag(array, loc_outfile, weights=[0.8,0.2],method="average"):
  if method == "average":
    scores = defaultdict(float)
  with open(loc_outfile,"wb") as outfile:
    for i, glob_file in enumerate(array):
      print "parsing:", i,glob_file
      # sort glob_file by first column, ignoring the first line
      lines = open(glob_file).readlines()
      lines = [lines[0]] + sorted(lines[1:])
      for e, line in enumerate( lines ):
        if i == 0 and e == 0:
          outfile.write(line)
        if e > 0:
          row = line.strip().split(",")
          scores[(e,row[0])] += float(row[1])*weights[i]
    for j,k in sorted(scores):
      outfile.write("%s,%f\n"%(k,(scores[(j,k)])))
    print("wrote to %s"%loc_outfile)
array=[ os.getcwd()+'/En/final.csv', os.getcwd()+'/En/blending.csv']

kaggle_bag(array, loc_outfile)

parsing: 0 /Users/shashank/Downloads/PS 3/En/final.csv
parsing: 1 /Users/shashank/Downloads/PS 3/En/blending.csv
wrote to out.csv


In [24]:
a=2
b=3
str(a)+str(b)

'23'

In [32]:
train['MGR_ID'].mean()

25988.957978577313

In [78]:
from sklearn.decomposition import TruncatedSVD
from sklearn.random_projection import sparse_random_matrix
X = sparse_random_matrix(100, 100, density=0.01, random_state=42)
svd = TruncatedSVD(n_components=5, random_state=42)
svd.fit(X) 

TruncatedSVD(algorithm='randomized', n_components=5, n_iter=5,
       random_state=42, tol=0.0)

In [18]:
test.shape

(58921, 140)